In [164]:
import pandas as pd
import numpy as np

In [165]:
data=pd.read_csv('/kaggle/input/food-data/data_recommender.csv')
data.head()

,Unnamed: 0,client_id,Food
0,0,ahNzfnNldmVucm9vbXMtc2VjdXJlciQLEhpuaWdodGxvb3...,"['Focaccia', 'Bucatini Cacio e Pepe', 'Pomodor..."
1,1,ahNzfnNldmVucm9vbXMtc2VjdXJlciQLEhpuaWdodGxvb3...,"['Gl Miraly Rose', 'The ""etta"" Chopped Salad',..."
2,2,ahNzfnNldmVucm9vbXMtc2VjdXJlciQLEhpuaWdodGxvb3...,"['Mimosa Package', ""Kid's French Toast"", 'Foca..."
3,3,ahNzfnNldmVucm9vbXMtc2VjdXJlciQLEhpuaWdodGxvb3...,"['Focaccia', 'Gl Lucarelli Primitivo', 'Spicy ..."
4,4,ahNzfnNldmVucm9vbXMtc2VjdXJlciQLEhpuaWdodGxvb3...,"['Notorious N.O.G', 'Sparrow Regular Coffee', ..."


In [166]:
data['Food']=data['Food']
data['Food']


0        ['Focaccia', 'Bucatini Cacio e Pepe', 'Pomodor...
1        ['Gl Miraly Rose', 'The "etta" Chopped Salad',...
2        ['Mimosa Package', "Kid's French Toast", 'Foca...
3        ['Focaccia', 'Gl Lucarelli Primitivo', 'Spicy ...
4        ['Notorious N.O.G', 'Sparrow Regular Coffee', ...
                               ...                        
66488    ['Etta Rita', 'Arnold Palmer', 'Huevos Rancher...
66489    ['Etta Rita', 'Johnnie Walker Black Label', 'G...
66490    ['Fromage Noir', 'Etta Old Fashioned', 'Caesar...
66491    ['Crispy Chicken Sandwich', 'Brunch Wild Mushr...
66492    ['Iced Tea', 'La Colombe Regular Coffee', 'Kou...
Name: Food, Length: 66493, dtype: object

In [167]:
import ast


def convert(text):
    l=[]
    
    for i in ast.literal_eval(text):
        if i not in l:
            temp=i.split()
#             print(temp)
            
            words="".join(temp)
            l.append(words)
            
            
    return l

print(convert("['Focaccia', 'Bucatini Cacio e Pepe', 'Pomodoro', 'Pomodoro']"))

['Focaccia', 'BucatiniCacioePepe', 'Pomodoro']


Making the global recommendation system api

In [168]:
global_food_list=[]

In [169]:
data['Food']=data['Food'].apply(convert)

In [170]:
data['Food']

0                 [Focaccia, BucatiniCacioePepe, Pomodoro]
1        [GlMiralyRose, The"etta"ChoppedSalad, WildMush...
2        [MimosaPackage, Kid'sFrenchToast, Focaccia, Po...
3        [Focaccia, GlLucarelliPrimitivo, SpicyMeatball...
4        [NotoriousN.O.G, SparrowRegularCoffee, GreenWi...
                               ...                        
66488    [EttaRita, ArnoldPalmer, HuevosRancheros, Etta...
66489    [EttaRita, JohnnieWalkerBlackLabel, GLOmero, C...
66490    [FromageNoir, EttaOldFashioned, Caesar, Diavol...
66491    [CrispyChickenSandwich, BrunchWildMushroom, St...
66492    [IcedTea, LaColombeRegularCoffee, Kouign-A, St...
Name: Food, Length: 66493, dtype: object

Now removing the 

In [171]:
import re

pattern = r'[0-9]'

for item in data['Food']:
    for small_item in item:
        small_item=small_item.lower().replace("'","").replace('"',"")
        small_item = re.sub(pattern, '',small_item)
        if small_item not in global_food_list:
            global_food_list.append(small_item)

In [196]:
print(global_food_list[:1000])

['focaccia', 'bucatinicacioepep', 'pomodoro', 'glmiralyro', 'theettachoppedsalad', 'wildmushroom', 'mimosapackag', 'kidsfrenchtoast', 'potatohash', 'elysiantarteflamb', 'crispychickensandwich', 'gllucarelliprimitivo', 'spicymeatb', 'grilledasparagu', 'lobstermalfadin', 'mushroomtortellini', 'celebrationshot', 'notoriousn.o.g', 'sparrowregularcoff', 'greenwithenvi', 'fig&coppapizza', 'pepperonipizza', 'sneakyfox', 'cosmonaut', 'margaritasforeveryon', 'wood-firedhangersteak', 'rosemaryfri', 'allinonsavori', 'mimosa', 'pastryplatt', 'ettasoftscrambl', 'shortribhash', 'bacon', 'ccardconveni', 'ettarita', 'btldomaineericlouissb', 'ricottapillow', 'cavatellibologn', 'bubblingshrimp', 'sunsetondamen', 'spicygreenbean', 'cultclass', 'firepi', 'lemonad', 'sprite', 'orecchiett', 'ettaoldfashion', 'naughtyboy', 'fortalezablanco', 'millerhighlif', 'arretereposado', 'fourro', 'pigpicn', 'margherita', 'cappelletti', 'bologn', 'caesar', 'farmgreensalad', 'charredbroccoli', 'macallandoublecask', 'wagy

In [173]:
len(global_food_list)

2022

In [187]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)     
global_food_list=list(map(stem,global_food_list))
# print(global_food_list)

In [174]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=1000,stop_words='english')

In [175]:
vectors=cv.fit_transform(global_food_list).toarray()

In [176]:
cv.get_feature_names_out()[:10]

array(['abevpackage', 'agedribeye', 'ahituna', 'almondrhubarb',
       'artichokepizza', 'askur', 'asparagus', 'banhez', 'blaumbros',
       'bolognese'], dtype=object)

In [177]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [183]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)
print(similarity)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [182]:
def recommend(food):
    food=food.lower().replace("'","").replace('"',"").replace(" ","")
    food = re.sub(pattern, '',food)
    food_index=global_food_list.index(food)
    distances=similarity[food_index]
    foods_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]
    print(foods_list)
    for i in foods_list:
        print(global_food_list[i[0]])

recommend("Caesar")
    

[(1479, 1.0), (0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]
e-caesar
focaccia
bucatinicacioepepe
pomodoro
glmiralyrose
theettachoppedsalad
wildmushroom
mimosapackage
kidsfrenchtoast
potatohash


In [180]:
recommend("SparrowRegularCoffee")

[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0)]
focaccia
bucatinicacioepepe
pomodoro
glmiralyrose
theettachoppedsalad


In [198]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect=TfidfVectorizer()

tfidf_jobid=tfidf_vect.fit_transform((global_food_list))
print(tfidf_jobid[3])

  (0, 893)	1.0


In [193]:
user_tfid=tfidf_vect.transform(global_food_list)
cos_similarity_df=map(lambda x:cosine_similarity(user_tfid,x),tfidf_jobid)
print(cos_similarity_df)

In [199]:
def recommend(food):
    food=food.lower().replace("'","").replace('"',"").replace(" ","")
    food = re.sub(pattern, '',food)
    food_index=global_food_list.index(food)
    distances=cos_similarity_df[food_index]
    foods_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]
    print(foods_list)
    for i in foods_list:
        print(global_food_list[i[0]])

recommend("Caesar")

TypeError: 'map' object is not subscriptable

In [205]:
!pip install py-translate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.2 MB/s eta 0:00:00


In [206]:
!pip install googletrans

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=91a70d1c0dca2812b6a3aa06be43bb626c7e87af3a2e3407f6412537e2288ad2
  Stored in directory: /root/.cache/pip/wheels/cd/15/16/9d24ab82fc6498f8150e1b478c1891e7139afc854f00b8af5d
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.1

In [208]:
from googletrans import Translator

translator = Translator()
result = translator.translate('Mikä on nimesi', src='fi', dest='fr')

print(result.text)

AttributeError: 'NoneType' object has no attribute 'group'

In [210]:
translate en zh-TW  'Hello World!'

SyntaxError: invalid syntax (1191066715.py, line 1)